## **Проект 1**. Исследование поведения пользователей
#### *Игра Quiz Freeze*

### Задачи проекта - проверить следующие гипотезы:
- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

In [2]:
import pandas as pd

In [3]:
event_df = pd.read_csv('data/7_4_Events.csv')
event_df.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [4]:
mask_for_year = (event_df['event_type'] == 'registration') & (event_df['start_time'] < '2019-01-01') & (event_df['start_time'] >= '2018-01-01')
registered_2018 = event_df[mask_for_year].user_id.values
event18_df = event_df[event_df['user_id'].isin(registered_2018)]
event18_df.head(10)


,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01T03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01T04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01T08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01T11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01T13:28:07,NaN,27836
51410,80313,registration,NaN,2018-01-01T14:08:40,NaN,27837
51411,80314,registration,NaN,2018-01-01T14:42:58,NaN,27838
51412,80315,tutorial_start,NaN,2018-01-01T14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01T15:00:51,31506.0,27835
51414,80317,tutorial_finish,NaN,2018-01-01T15:06:15,31506.0,27835


In [5]:
purchase_df = pd.read_csv('data/purchase.csv')
purchase_df.head()

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


In [6]:
purchase18_df = purchase_df[purchase_df['user_id'].isin(registered_2018)]
purchase18_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


In [7]:
#Объединяем таблицы событий и покупок

purchase18_df['event_type'] = 'purchase'

event18_df = event18_df.rename(columns = {'id': 'event_id'})
purchase18_df = purchase18_df.rename(columns = {'id': 'purchase_id', 'event_datetime':'start_time'})

total_events18 = pd.concat([event18_df, purchase18_df])

#сбрасываем индексы в объединенной таблице
total_events18 = total_events18.reset_index(drop=True).sort_values('start_time')
total_events18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 0 to 68558
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event_id        66959 non-null  float64
 1   event_type      68559 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      68559 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         68559 non-null  int64  
 6   purchase_id     1600 non-null   float64
 7   amount          1600 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 4.7+ MB


C:\Users\Анастасия\AppData\Local\Temp\ipykernel_8308\849561813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase18_df['event_type'] = 'purchase'


In [8]:
#Преобразуем тип данных 
total_events18['start_time'] = pd.to_datetime(total_events18['start_time'])
total_events18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 0 to 68558
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      68559 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   amount          1600 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 4.7+ MB


In [9]:
total_events18['selected_level'].value_counts()

medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

##### Группа пользователей, выбравших уровень "easy"

In [55]:
users_easy_level = total_events18[total_events18['selected_level'] == 'easy']['user_id'].unique()
print("Количество пользователей, выбравших уровень easy:", len(users_easy_level))
print("Процент пользователей, выбравших уровень easy, от общего количества пользователей: {:.2%}".format(len(users_easy_level)/len(registered_2018)))

Количество пользователей, выбравших уровень easy: 2448
Процент пользователей, выбравших уровень easy, от общего количества пользователей: 12.29%


In [47]:
easy_level = total_events18[total_events18['user_id'].isin(users_easy_level)]
users_easy_level_purchase = easy_level[easy_level['event_type'] == 'purchase']['user_id'].unique()
print("Количество пользователей, выбравших уровень easy и оплативших впоследствии пакет вопросов:", len(users_easy_level_purchase))

Количество пользователей, выбравших уровень easy и оплативших впоследствии пакет вопросов: 189


In [12]:
percent_of_purchase_easy = len(users_easy_level_purchase) / len(users_easy_level)
print(
    "Процент оплат среди пользователей, выбравших уровень easy: {:.2%}".format(percent_of_purchase_easy)
)

Процент оплат среди пользователей, выбравших уровень easy: 7.72%


##### Группа пользователей, выбравших уровень "medium"

In [56]:
users_medium_level = total_events18[total_events18['selected_level'] == 'medium']['user_id'].unique()
print("Количество пользователей, выбравших уровень medium:", len(users_medium_level))
print("Процент пользователей, выбравших уровень medium, от общего количества пользователей: {:.2%}".format(len(users_medium_level)/len(registered_2018)))

Количество пользователей, выбравших уровень medium: 4645
Процент пользователей, выбравших уровень medium, от общего количества пользователей: 23.31%


In [49]:
medium_level = total_events18[total_events18['user_id'].isin(users_medium_level)]
users_medium_level_purchase = medium_level[medium_level['event_type'] == 'purchase']['user_id'].unique()
print("Количество пользователей, выбравших уровень medium и оплативших впоследствии пакет вопросов:", len(users_medium_level_purchase))

Количество пользователей, выбравших уровень medium и оплативших впоследствии пакет вопросов: 969


In [15]:
percent_of_purchase_medium = len(users_medium_level_purchase) / len(users_medium_level)
print(
    "Процент оплат среди пользователей, выбравших уровень medium: {:.2%}".format(percent_of_purchase_medium)
)

Процент оплат среди пользователей, выбравших уровень medium: 20.86%


##### Группа пользователей, выбравших уровень "hard"

In [57]:
users_hard_level = total_events18[total_events18['selected_level'] == 'hard']['user_id'].unique()
print("Количество пользователей, выбравших уровень hard:", len(users_hard_level))
print("Процент пользователей, выбравших уровень hard, от общего количества пользователей: {:.2%}".format(len(users_hard_level)/len(registered_2018)))

Количество пользователей, выбравших уровень hard: 1249
Процент пользователей, выбравших уровень hard, от общего количества пользователей: 6.27%


In [51]:
hard_level = total_events18[total_events18['user_id'].isin(users_hard_level)]
users_hard_level_purchase = hard_level[hard_level['event_type'] == 'purchase']['user_id'].unique()
print("Количество пользователей, выбравших уровень hard и оплативших впоследствии пакет вопросов:", len(users_hard_level_purchase))

Количество пользователей, выбравших уровень hard и оплативших впоследствии пакет вопросов: 442


In [18]:
percent_of_purchase_hard = len(users_hard_level_purchase) / len(users_hard_level)
print(
    "Процент оплат среди пользователей, выбравших уровень hard: {:.2%}".format(percent_of_purchase_hard)
)

Процент оплат среди пользователей, выбравших уровень hard: 35.39%


**Вывод.**  
Сегментирование пользователей на группы по выбранному ими уровню сложности позволяет сделать следующие выводы:
- чем сложнее выбранный пользователем уровень, тем выше вероятность совершения им оплаты пакета вопросов;
- несмотря на то, что уровень hard выбирают всего 6% всех зарегистрированных, вероятность совершения оплаты этими пользователями выше; 
- наиболее популярный уровень среди пользователей - medium - его выбирают 23% всех пользователей, 21% из которых впоследствии совершает оплату.

##### Анализ временных промежутков

In [23]:
#Вынесем данные о времени регистрации пользователей в отдельную таблицу
registration_df = total_events18[total_events18['event_type'] == 'registration'][['user_id', 'start_time']].rename(columns = {'start_time':'registration_time'})
registration_df

,user_id,registration_time
0,27832,2018-01-01 03:48:40
1,27833,2018-01-01 04:07:25
2,27834,2018-01-01 08:35:10
3,27835,2018-01-01 11:54:47
4,27836,2018-01-01 13:28:07
...,...,...
66941,47753,2018-12-31 18:58:55
66942,47754,2018-12-31 19:14:08
66947,47755,2018-12-31 21:15:14
66950,47756,2018-12-31 23:17:30


In [26]:
#Вынесем данные о времени совершения покупки пользователем в отдельную таблицу
purchase_time_df = total_events18[total_events18['event_type'] == 'purchase'][['user_id', 'start_time']].rename(columns = {'start_time':'purchase_time'})
purchase_time_df

,user_id,purchase_time
66959,27845,2018-01-03 18:53:43
66960,27865,2018-01-04 14:46:10
66961,27911,2018-01-07 08:19:12
66962,27910,2018-01-07 12:11:34
66963,27940,2018-01-07 13:16:41
...,...,...
68554,47498,2019-01-02 03:48:19
68555,47647,2019-01-02 23:26:26
68556,47554,2019-01-03 00:36:36
68557,47742,2019-01-04 12:51:41


In [33]:
# Объединим данные о времени регистрации и времени совершения покупки по каждому пользователю в отдельную таблицу
total_delta_time_df = registration_df.merge(purchase_time_df, on='user_id', how = 'inner')
total_delta_time_df

,user_id,registration_time,purchase_time
0,27845,2018-01-02 01:35:56,2018-01-03 18:53:43
1,27865,2018-01-03 11:14:57,2018-01-04 14:46:10
2,27884,2018-01-04 11:50:43,2018-01-08 19:37:34
3,27910,2018-01-05 10:45:33,2018-01-07 12:11:34
4,27911,2018-01-05 10:48:24,2018-01-07 08:19:12
...,...,...,...
1595,47671,2018-12-28 20:48:59,2018-12-30 01:39:50
1596,47687,2018-12-29 09:49:24,2018-12-30 09:05:28
1597,47712,2018-12-29 18:21:28,2018-12-31 09:05:20
1598,47732,2018-12-30 14:20:21,2019-01-06 07:14:25


Среднее время между оплатой и регистрацией для пользователей, выбравших уровень easy

In [64]:
easy_delta_time = total_delta_time_df[total_delta_time_df['user_id'].isin(users_easy_level)]
easy_delta_time['delta_time'] = (easy_delta_time['purchase_time'] - easy_delta_time['registration_time'])
print('Среднее время между оплатой и регистрацией для пользователей, выбравших уровень easy:', easy_delta_time['delta_time'].mean())

Среднее время между оплатой и регистрацией для пользователей, выбравших уровень easy: 3 days 22:10:23.211640211


C:\Users\Анастасия\AppData\Local\Temp\ipykernel_8308\2688298990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  easy_delta_time['delta_time'] = (easy_delta_time['purchase_time'] - easy_delta_time['registration_time'])


Среднее время между оплатой и регистрацией для пользователей, выбравших уровень medium

In [66]:
medium_delta_time = total_delta_time_df[total_delta_time_df['user_id'].isin(users_medium_level)]
medium_delta_time['delta_time'] = medium_delta_time['purchase_time'] - medium_delta_time['registration_time']
print('Среднее время между оплатой и регистрацией для пользователей, выбравших уровень medium:', medium_delta_time['delta_time'].mean())

Среднее время между оплатой и регистрацией для пользователей, выбравших уровень medium: 4 days 06:12:06.576883384


C:\Users\Анастасия\AppData\Local\Temp\ipykernel_8308\773075277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium_delta_time['delta_time'] = medium_delta_time['purchase_time'] - medium_delta_time['registration_time']


Среднее время между оплатой и регистрацией для пользователей, выбравших уровень hard

In [67]:
hard_delta_time = total_delta_time_df[total_delta_time_df['user_id'].isin(users_hard_level)]
hard_delta_time['delta_time'] = hard_delta_time['purchase_time'] - hard_delta_time['registration_time']
print('Среднее время между оплатой и регистрацией для пользователей, выбравших уровень hard:', hard_delta_time['delta_time'].mean())

Среднее время между оплатой и регистрацией для пользователей, выбравших уровень hard: 3 days 14:55:19.257918552


C:\Users\Анастасия\AppData\Local\Temp\ipykernel_8308\1873550097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hard_delta_time['delta_time'] = hard_delta_time['purchase_time'] - hard_delta_time['registration_time']


**Вывод.**

Разница во времени между регистрацией и оплатой пакета вопросов не зависит от выбранного пользователем уровня сложности. В среднем пользователь оплачивает пакет вопросов примерно через 3-4 дня после регистрации.